In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import time
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
data = pd.read_csv('~/Desktop/716/javaperformance/c_sharp.csv')
data.head(5)

,QueryString,PredictedLabel
0,cs1513 c# } expected,False
1,cs2001 c# source file could not be found.,False
2,cs1001 c# identifier expected,False
3,ide0055 c# fix formatting,False
4,cs7036 c# there is no argument given that corr...,False


In [3]:
data["len"] = [len(text) for text in data["QueryString"].values]
data.groupby("PredictedLabel")["len"].mean()

PredictedLabel
False    66.705882
True     16.546422
Name: len, dtype: float64

In [4]:
data.head()

,QueryString,PredictedLabel,len
0,cs1513 c# } expected,False,20
1,cs2001 c# source file could not be found.,False,41
2,cs1001 c# identifier expected,False,29
3,ide0055 c# fix formatting,False,25
4,cs7036 c# there is no argument given that corr...,False,89


In [5]:
def cleanText(text):
    
    lemma = WordNetLemmatizer()
    stp = stopwords.words('english')
    
    text = re.sub("[^a-zA-Z0-9#]"," ",text)
    text = text.lower()

    text = nltk.word_tokenize(text)
    text = [lemma.lemmatize(word) for word in text]
    
    text = [word for word in text if word not in stp]
    
    text = " ".join(text)
    
    return text

In [6]:
cleanedText = []
for text in data["QueryString"]:   
    cleanedText.append(cleanText(text))
print(cleanedText[0])

cs1513 c # expected


In [7]:
data["PredictedLabel"].value_counts()

True     4039
False     935
Name: PredictedLabel, dtype: int64

In [8]:
y = data["PredictedLabel"]
y.shape

(4974,)

In [9]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(cleanedText).toarray()
x.shape

(4974, 1990)

In [10]:
print(np.histogram(x))

(array([9885059,     711,    1722,    1680,    1609,    1805,    1911,
          1494,     794,    1475], dtype=int64), array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))


In [11]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=4,test_size=0.3)
X_train_lab, X_test_unlab, y_train_lab, y_test_unlab = train_test_split(x_train, y_train, test_size=0.50, random_state=1, stratify=y_train)

In [22]:
#classifier = XGBClassifier(objective= 'reg:squaredlogerror', use_label_encoder=False)
#classifier = SVC()
classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100)
classifier.fit(x_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)

In [17]:
# y_pred = classifier.predict(x_test)
# score = accuracy_score(y_test, y_pred)
# print('Accuracy: %.3f' % (score*100))

In [23]:

#model = LogisticRegression()
#model = SVC()
#model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100)
#model.fit(X_train_lab, y_train_lab)
LogisticRegression = classifier

In [24]:
#yhat = model.predict(x_test)

#score = accuracy_score(y_test, yhat)
#print('Accuracy: %.3f' % (score*100))

In [25]:
nolabel = [-1 for _ in range(len(y_test_unlab))]

y_train_mixed = concatenate((y_train_lab, nolabel))

model = LabelPropagation()

model.fit(x_train, y_train)

yhat = model.predict(x_test)

score = accuracy_score(y_test, yhat)

print('Accuracy: %.3f' % (score*100))

Accuracy: 91.025


In [19]:
tran_labels = model.transduction_

In [26]:
#model2 = SVC()
model2 = LogisticRegression
model2.fit(x_train, tran_labels)

y_hat = model2.predict(x_test)
score2 = accuracy_score(y_test, y_hat)
print('Accuracy: %.3f' % (score2*100))

Accuracy: 92.431


In [ ]:
#x.astype(numpy.float32)

In [27]:
y_pred = y_hat.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)


MSE test data: 0.075687
RMSE test data: 0.52451
R2 test data: 0.46559
[[ 198   57]
 [  56 1182]]
198
57
56
1182
Accuracy =  0.9243134628265238
Precision =  0.7764705882352941
Recall =  0.7795275590551181
f1score =  0.7779960707269156


# Other single classifier models

In [37]:
model3 = SGDClassifier()
model3.fit(X_train_lab, y_train_lab)

y_hat3 = model3.predict(x_test)
score3 = accuracy_score(y_test, y_hat3)
print('Accuracy: %.3f' % (score3*100))

Accuracy: 90.489


In [38]:
y_pred = y_hat3.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)


MSE test data: 0.095111
RMSE test data: 0.55534
R2 test data: 0.32844
[[ 158   97]
 [  45 1193]]
158
97
45
1193
Accuracy =  0.9048894842598795
Precision =  0.6196078431372549
Recall =  0.7783251231527094
f1score =  0.6899563318777293


In [39]:
model4 = XGBClassifier(use_label_encoder=False, objective= 'reg:squaredlogerror')
model4.fit(X_train_lab, y_train_lab)

y_hat4 = model4.predict(x_test)
score4 = accuracy_score(y_test, y_hat4)
print('Accuracy: %.3f' % (score4*100))

Accuracy: 90.355


In [40]:
y_pred = y_hat4.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.09645
RMSE test data: 0.55728
R2 test data: 0.31898
[[ 169   86]
 [  58 1180]]
169
86
58
1180
Accuracy =  0.9035498995311454
Precision =  0.6627450980392157
Recall =  0.7444933920704846
f1score =  0.7012448132780082


In [43]:
from sklearn.linear_model import LogisticRegression
model5 = LogisticRegression()
model5.fit(X_train_lab, y_train_lab)

y_hat5 = model5.predict(x_test)
score5 = accuracy_score(y_test, y_hat5)
print('Accuracy: %.3f' % (score5*100))

Accuracy: 88.814


In [44]:
y_pred = y_hat5.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.11186
RMSE test data: 0.57831
R2 test data: 0.2102
[[  93  162]
 [   5 1233]]
93
162
5
1233
Accuracy =  0.8881446751507033
Precision =  0.36470588235294116
Recall =  0.9489795918367347
f1score =  0.5269121813031161


In [45]:
model7 = MultinomialNB()
model7.fit(X_train_lab, y_train_lab)

y_hat7 = model7.predict(x_test)
score7 = accuracy_score(y_test, y_hat7)
print('Accuracy: %.3f' % (score7*100))

Accuracy: 89.484


In [46]:
y_pred = y_hat7.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.10516
RMSE test data: 0.56946
R2 test data: 0.2575
[[ 109  146]
 [  11 1227]]
109
146
11
1227
Accuracy =  0.8948425987943738
Precision =  0.42745098039215684
Recall =  0.9083333333333333
f1score =  0.5813333333333334


In [47]:
model8 = DecisionTreeClassifier()
model8.fit(X_train_lab, y_train_lab)

y_hat8 = model8.predict(x_test)
score8 = accuracy_score(y_test, y_hat8)
print('Accuracy: %.3f' % (score8*100))

Accuracy: 90.958


In [48]:
y_pred = y_hat8.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.090422
RMSE test data: 0.54836
R2 test data: 0.36154
[[ 191   64]
 [  71 1167]]
191
64
71
1167
Accuracy =  0.9095780308104487
Precision =  0.7490196078431373
Recall =  0.7290076335877863
f1score =  0.7388781431334622
